In [ ]:
import torch
import transformers

# import
import openai

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import OpenAIEmbedding

from langchain.llms import Ollama
import chromadb

from dotenv import dotenv_values

In [ ]:
api_key = dotenv_values('.env')["OPENAI_API_KEY"]
openai.api_key = api_key

In [ ]:
#Set the embedding

llm = Ollama(model="llama2")
#embed_model = OllamaEmbeddings(base_url="http://localhost:11434", model="llama2") #Local Llama 2 embedding model
embed_model = OpenAIEmbedding() #Using OpenAI's text-embed-002

In [ ]:
COLLECTION = "aiprof"
PATH = './chroma'

In [ ]:
# create client and a new collection
db = chromadb.PersistentClient(path=PATH)
chroma_collection = db.get_or_create_collection(COLLECTION)

In [ ]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

In [ ]:
# load documents
documents = SimpleDirectoryReader("data").load_data()

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

In [ ]:
# load from disk
db2 = chromadb.PersistentClient(path=PATH)
chroma_collection = db2.get_or_create_collection(COLLECTION)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index2 = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [ ]:
query_engine = index2.as_query_engine()

In [ ]:
resp = query_engine.query("What is The rational agent approach?")
print(resp.response)

In [28]:
resp = query_engine.query("Generate 3 concise questions from the PDFs")


In [29]:
resp.response.split('\n')

['Sure! Based on the provided context information, here are three concise questions that could be asked related to the topic of AI systems:',
 '',
 '1. How do AI researchers map theoretical knowledge to develop special forms (cookbook recipes) for solving complex problems? Can you provide an example of such a mapping?',
 '2. What is the significance of DENDRAL, a knowledge-based system developed in the 1960s and 1970s? How did it differ from other AI systems at that time?',
 '3. How do you determine the truth value of a sentence in a partial model, and why is it not possible to determine it efficiently in general? Can you provide an example of such a situation?']